In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.models import Model

Using TensorFlow backend.


ImportError: No module named tensorflow

In [4]:
tmp = np.load('C:/Users/saite/Desktop/Stats and CT Lab/Statistics and Centr tend Lab prob/Deep Learning9917/cifar_pca_train.npz')
train_data = tmp['data']
train_labels = tmp['labels']

tmp = np.load('C:/Users/saite/Desktop/Stats and CT Lab/Statistics and Centr tend Lab prob/Deep Learning9917/cifar_pca_test.npz')
test_data = tmp['data']
test_labels = tmp['labels']

In [ ]:
# Training an autoencoder model on cifar-10 PCA reduced data

input_img = Input(shape=(781,))
crrpt_img = Dropout(0.5)(input_img)
encoded = Dense(1000, activation='sigmoid')(crrpt_img)
decoded = Dense(781, activation='linear')(encoded)

autoencoder = Model(input_img,decoded)

nb_epoch = 50
batch_size = 32

autoencoder.compile(optimizer='adam',
                    loss='mean_squared_error',
                    metrics=['mse'])


In [5]:
history = autoencoder.fit(train_data, train_data,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_data, test_data),
                    verbose=1)


autoencoder.save('DAE_l1_model.h5')

NameError: name 'autoencoder' is not defined

In [ ]:
encoder = Model(input_img,encoded)
htrain_data = encoder.predict(train_data)


In [ ]:
input_img1 = Input(shape=(1000,))
crrpt_img1 = Dropout(0.5)(input_img1)
encoded1 = Dense(1000, activation='sigmoid')(crrpt_img1)
decoded1 = Dense(1000, activation='sigmoid')(encoded1)

autoencoder1 = Model(input_img1,decoded1)

autoencoder1.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['binary_crossentropy'])

In [ ]:
history = autoencoder1.fit(htrain_data, htrain_data,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    verbose=1)


autoencoder1.save('DAE_l2_model.h5')

In [ ]:
# Converting labels into one-hot vectors for MLP training

train_labels = keras.utils.to_categorical(train_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)

In [ ]:
# training a simple one hidden layer MLP for classification task

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(781,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))



mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = mlp.fit(train_data[:10000], train_labels[:10000],
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(test_data, test_labels))

In [ ]:
# Training with unsupervised initialization for layer-1 
# of an MLP using autoencoder weights

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(781,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))



mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

mlp.layers[1].set_weights(autoencoder.layers[2].get_weights())
mlp.layers[3].set_weights(autoencoder1.layers[2].get_weights())


history = mlp.fit(train_data[:10000], train_labels[:10000],
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(test_data, test_labels))

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15191092183439573984
]
